In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

In [40]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import sklearn.metrics as metrics

# Cargar data

- cargo

In [41]:
path1 = "./dataset.csv"

In [42]:
path = './dataset_clean.csv'

In [43]:
# from google.colab import drive
# drive.mount('/content/drive')

In [44]:
df= pd.read_csv(path)

In [45]:
df.drop(columns = ["a [ang]"], axis=[1], inplace = True)

# Preprocesamiento

- Resuelvo problemas con descriptores que tienen infinitos

In [46]:
filter = df['Vacancy energy (log[eV/O atom])'] == -np.inf
df['Vacancy energy (log[eV/O atom])'][filter] = np.nan

<ipython-input-46-25d05f01ba52>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Vacancy energy (log[eV/O atom])'][filter] = np.nan


In [47]:
filter = df['Stability (log[eV/atom])'] == -np.inf
df['Stability (log[eV/atom])'][filter] = np.nan

<ipython-input-47-6eee1e0e4a1f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Stability (log[eV/atom])'][filter] = np.nan


- Elimino nulos sí o sí (la otra alternativa sería imputar)

In [48]:
df.isnull().sum()

Unnamed: 0                            0
Lowest distortion                     0
A                                     0
B                                     0
Radius A [ang]                        0
Radius B [ang]                        0
Formation energy [eV/atom]            0
Stability [eV/atom]                   0
Volume per atom [A^3/atom]            0
Band gap [eV]                         0
Vacancy energy [eV/O atom]          349
Stability (log[eV/atom])            290
Vacancy energy (log[eV/O atom])    3169
Magnetic moment (^2[mu_B])          947
Valence B_1                           0
Valence B_2                           0
Valence B_3                           0
Valence B_4                           0
Valence B_5                           0
Valence B_element not in BV           0
Valence B_not balanced                0
dtype: int64

In [49]:
df.drop('Vacancy energy (log[eV/O atom])', axis=1, inplace= True)

In [50]:
df.dropna(inplace=True)

# Split target y patrón

- Split

In [51]:
x = df[['Radius A [ang]','Radius B [ang]','Formation energy [eV/atom]', 'Stability [eV/atom]', 'Volume per atom [A^3/atom]','Vacancy energy [eV/O atom]', 'Stability (log[eV/atom])']]
y = df['Lowest distortion']



In [52]:
x.columns

Index(['Radius A [ang]', 'Radius B [ang]', 'Formation energy [eV/atom]',
       'Stability [eV/atom]', 'Volume per atom [A^3/atom]',
       'Vacancy energy [eV/O atom]', 'Stability (log[eV/atom])'],
      dtype='object')

- Validación

In [53]:
print(x.shape)
print(y.shape)

(3720, 7)
(3720,)


# Split train y test

- Reducimos la dimensionalidad

In [54]:
cols_str = x.columns
cols_str

Index(['Radius A [ang]', 'Radius B [ang]', 'Formation energy [eV/atom]',
       'Stability [eV/atom]', 'Volume per atom [A^3/atom]',
       'Vacancy energy [eV/O atom]', 'Stability (log[eV/atom])'],
      dtype='object')

In [55]:
X_cols = cols_str
y_col = ['Lowest distortion']

x = df[X_cols]
y = df[y_col]

- Split

In [56]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42, shuffle = True)

- Validación

In [57]:
print(X_train.shape)
print(X_test.shape)
print('')
print(y_train.shape)
print(y_test.shape)

(2976, 7)
(744, 7)

(2976, 1)
(744, 1)


# Normalización de los datos

- Instancio scaler para target y patron

In [58]:
x_st_scaler = StandardScaler() #Creo instancia del scaler
y_st_scaler = StandardScaler() #Creo instancia del scaler

In [59]:
X_train = x_st_scaler.fit_transform(X_train) # escalo train
X_test = x_st_scaler.transform(X_test) #transformo test

In [60]:
X_test.shape

(744, 7)

# Entrenamiento del modelo Random Forest

- Importo modelo de Sklearn

In [61]:
#from sklearn.tree import DecisionTreeClassifier, RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree

- Instancio modelo

In [62]:
modelTree=RandomForestClassifier(n_estimators=130, criterion="entropy",max_depth=12, min_samples_split=3, max_features ="log2", random_state=3)
modelTree

RandomForestClassifier(criterion='entropy', max_depth=12, max_features='log2',
                       min_samples_split=3, n_estimators=130, random_state=3)

- Entreno

In [63]:
modelTree.fit(X_train, y_train)

<ipython-input-63-0caf4761789b>:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  modelTree.fit(X_train, y_train)


RandomForestClassifier(criterion='entropy', max_depth=12, max_features='log2',
                       min_samples_split=3, n_estimators=130, random_state=3)

- Predigo

In [64]:
y_pred = modelTree.predict(X_test)
y_pred.shape

(744,)

- Valido


In [65]:
from sklearn.metrics import matthews_corrcoef

coeficiente_mcc = matthews_corrcoef(y_test, y_pred)
print("Random Forest coeficiente de Matthews :", coeficiente_mcc)

Random Forest coeficiente de Matthews : 0.8641704864063664


In [66]:
rl_predictions =modelTree.predict(X_test)


In [67]:
np.unique(y)

array([0, 1, 2, 3])

In [68]:
y_test.shape

(744, 1)

In [69]:
#df_raw=pd.read_csv("./dataset.csv")
df_raw= pd.read_csv(path1)


# Deploy

- Imports

In [70]:
from ipywidgets import Dropdown, FloatSlider, IntSlider, interact, FloatText,fixed, interact_manual
import ipywidgets as widgets
import seaborn as sns

In [71]:
def get_radius(elementA, elementB):

    filterA = df_raw['A'] == elementA
    radiusA = df_raw[filterA]['Radius A [ang]'].median()

    filterB = df_raw['B'] == elementB
    radiusB = df_raw[filterB]['Radius B [ang]'].median()

    return[radiusA, radiusB]

In [77]:
def make_predictions(elementA, elementB, formationEnergy, stability, volume, vacancyEnergy):

    radius = get_radius(elementA, elementB) #obtengo radios
    data_new = radius + [formationEnergy, stability, volume, vacancyEnergy,np.log2(stability)] #concateno los datos
    data_df = pd.DataFrame(columns=cols_str)
    data_df.loc[0] = data_new

    #print(data_df)

    s_data = x_st_scaler.transform(data_df)#escalo data
    prediction = modelTree.predict(s_data) #Predigo
    #imprimo resultados

    if prediction==0:
      predicted_structure =  'cubic'
    if prediction==1:
      predicted_structure =  'orthorombic'
    if prediction==2:
      predicted_structure =  'rhombohedral'
    else:
      predicted_structure =  'tetragonal'

    return f"Predictd crystal structure  {predicted_structure}"



In [73]:
get_radius('Ag','Ag')

[1.28, 0.95]

In [74]:

result=make_predictions('Ac','Bi',-2.5, 0.27, 17.182, 2.563)
result

   Radius A [ang]  Radius B [ang]  Formation energy [eV/atom]  \
0            1.12             0.9                        -2.5   

   Stability [eV/atom]  Volume per atom [A^3/atom]  \
0                 0.27                      17.182   

   Vacancy energy [eV/O atom]  Stability (log[eV/atom])  
0                       2.563                 -1.888969  


'Predictd crystal structure  tetragonal'

In [75]:
x.columns

Index(['Radius A [ang]', 'Radius B [ang]', 'Formation energy [eV/atom]',
       'Stability [eV/atom]', 'Volume per atom [A^3/atom]',
       'Vacancy energy [eV/O atom]', 'Stability (log[eV/atom])'],
      dtype='object')

In [78]:
interact(
    #funcion a ejecutar con la data que recibe
    make_predictions,
    elementA = Dropdown(options =  list(df_raw['A'].unique()) ),
    elementB = Dropdown(options =  list(df_raw['B'].unique()) ),
    formationEnergy = widgets.FloatSlider(min = x['Formation energy [eV/atom]'].min(), max= x['Formation energy [eV/atom]'].max(), step = 0.001, value = x['Formation energy [eV/atom]'].mean()),
    # variables, recibidas de sliders, le digo que por defecto reciba la medi
    stability = widgets.FloatSlider(min = x['Stability [eV/atom]'].min(), max= x['Stability [eV/atom]'].max(), step = 0.001, value = x['Stability [eV/atom]'].mean()),
    volume = widgets.FloatSlider(min = x['Volume per atom [A^3/atom]'].min(), max= x['Volume per atom [A^3/atom]'].max(), step = 0.001, value = x['Volume per atom [A^3/atom]'].mean()),
    vacancyEnergy = widgets.FloatSlider(min = x['Vacancy energy [eV/O atom]'].min(), max= x['Vacancy energy [eV/O atom]'].max(), step = 0.001, value = x['Vacancy energy [eV/O atom]'].mean())
);

interactive(children=(Dropdown(description='elementA', options=('Ac', 'Ag', 'Al', 'As', 'Au', 'B', 'Ba', 'Be',…